In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import Machine, generate_objects, BunchTracker, track, compare_traces
from test_tools import compare_projections, plot_debug_data

from PyHEADTAIL.feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL.feedback.processors.multiplication import ChargeWeighter
from PyHEADTAIL.feedback.processors.linear_transform import Averager
from PyHEADTAIL.feedback.processors.misc import Bypass
from PyHEADTAIL.feedback.processors.register import Register, TurnDelay
from PyHEADTAIL.feedback.processors.convolution import Lowpass, Gaussian, FIRFilter
from PyHEADTAIL.feedback.processors.resampling import DAC, ADC
np.random.seed(0)

In [ ]:
""" 
    This test demonstrates how semi-realistic bunch by bunc feedback system can be implemented
"""

In [ ]:
# Basic parameters for the simulation
n_macroparticles = 100000
n_slices = 50
n_segments = 5
n_sigma_z = 4

# Longitudinal motion of the bunch is not taken into account in this example.
machine = Machine(n_segments= n_segments)
Q_x = machine.accQ_x
Q_y = machine.accQ_y

circumference = machine.circumference
h_RF = machine.h_RF

bunch_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles, 
                                                             n_slices,n_sigma_z)


In [ ]:
# General feedback parameters
feedback_gain = 0.1
# feedback_gain = (0.1,0.4)

# Parameters for the registers
delay = 1 
n_values = 2

# A number of turns to be tracked
n_turns = 50


# feedback settings
fc=1e10 # The cut off frequency of the power amplifier


In [ ]:
oscillation_wavelength = 1./2e9*c # [m]
oscillation_amplitide = 1e-3 # [m]

map_min = (bunch_ref.z > -80)
map_max = (bunch_ref.z < -40)
map_total = map_min*map_max

# bunch_ref.x[map_total] = bunch_ref.x[map_total]+1e-3
# bunch_ref.y[map_total] = bunch_ref.y[map_total]+1e-3

# bunch_ref.x += oscillation_amplitide*np.sin(2*np.pi*bunch_ref.z/oscillation_lambda)
# bunch_ref.y += oscillation_amplitide*np.sin(2*np.pi*bunch_ref.z/oscillation_lambda)
bunch_ref.x += oscillation_amplitide*np.cos(2*np.pi*bunch_ref.z/oscillation_wavelength)
bunch_ref.y += oscillation_amplitide*np.cos(2*np.pi*bunch_ref.z/oscillation_wavelength)
    
bunch_init = copy.deepcopy(bunch_ref)
tracker_ref = BunchTracker(bunch_init)
maps_ref = [i for i in trans_map]
# + [long_map]
track(n_turns, bunch_init,maps_ref,tracker_ref)

In [ ]:
# A reference feedback system, which is originally introduced in the file 001_ideal_feedbacks.ipynb:

bunch_OneBox_bunch = copy.deepcopy(bunch_ref)
tracker_OneBox_bunch = BunchTracker(bunch_OneBox_bunch)
slicer_OneBox_bunch = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_OneBox_bunch,processors_bunch_x,processors_bunch_y)
total_map_OneBox_bunch = [i for i in trans_map] + [feedback_map]
#  + [long_map]

track(n_turns, bunch_OneBox_bunch,total_map_OneBox_bunch,tracker_OneBox_bunch)

In [ ]:
# In this test, the ideal BPM signal is digitized by using one sample per bunch
# Then the signal is delayed one turn and betatron phase correction is applied by 
# using a vector sum algorithm. After the DAC the bandwidth is limited by using
# a Gaussian filter

bunch_built_in = copy.deepcopy(bunch_ref)
tracker_built_in = BunchTracker(bunch_built_in)
slicer_built_in = copy.deepcopy(slicer_ref)

processors_built_in_x = [
        Bypass(debug=True),
        ChargeWeighter(debug=True),
        Gaussian(fc, debug=True),
        TurnDelay(delay, Q_x, n_values, debug=True)
]

processors_built_in_y = [
        Bypass(debug=True),
        ChargeWeighter(debug=True),
        Gaussian(fc, debug=True),
        TurnDelay(delay, Q_y, n_values, debug=True),
]


feedback_map = OneboxFeedback(feedback_gain,slicer_built_in,
                              processors_built_in_x, processors_built_in_y, mpi = True)
total_map_built_in = [feedback_map] + [i for i in trans_map]
#  + [long_map]

track(n_turns, bunch_built_in,total_map_built_in,tracker_built_in)
plot_debug_data(processors_built_in_x, source = 'output')

In [ ]:
# In here, the traces and the projections from different implementations of the feedback system are compared.
# Note: the properties are calculated over all particles in all bunches
# Note: the scale in the emittance figures

compare_traces([tracker_OneBox_bunch,tracker_built_in],
               ['Ideal', 'Built in function', 'Manual FIR filters'])
compare_projections([ bunch_OneBox_bunch,  bunch_built_in], 
                    ['Ideal', 'Built in function', 'Manual FIR filters'])

# compare_traces([tracker_OneBox_bunch,tracker_register_example],
#                ['Ideal', 'Delayed', 'Separated'])
# compare_projections([ bunch_OneBox_bunch,  bunch_register_example], 
#                     ['Ideal', 'Delayed', 'Separated'])

Jani Komppula, CERN, 2017